<a href="https://colab.research.google.com/github/afaq5587/Lang-chain/blob/main/Rag_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade langchain-text-splitters langchain-community langchain-google-genai  langchain-pinecone

!pip install tqdm
!pip install -qU pypdf
!pip install langchain
!pip install langchain-google-genai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 8.0 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from google.colab import drive
from langchain_pinecone import PineconeVectorStore
from pinecone import ServerlessSpec,Pinecone

In [ ]:
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-flash")
drive.mount('/content/drive')

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings.embed_query("Hello world")

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = TextLoader("documents.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print(docs)

In [ ]:
from tqdm import tqdm

vectors=[]
for doc in tqdm(docs):
    vector = embeddings.embed_query(doc.page_content)
    doc_id = str(hash(doc.page_content))
    metadata = {
    "text": doc.page_content,
    }
    index.upsert(vectors=[(doc_id, vector, metadata)])

In [ ]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone,ServerlessSpec

index_name = "starta"
pc = Pinecone(api_key= PINECONE_API_KEY)
indexes = pc.list_indexes()

if index_name in indexes:
    print(f"Index '{index_name}' already exists. Choose a different name.")
else:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
index = pc.Index(index_name)
vector_store = PineconeVectorStore(embedding=embeddings, index=index, text_key="text")


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("documents.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)
print(docs)

[Document(metadata={'source': 'documents.txt'}, page_content='Abdul Qadeer Khan (1936–2021) was a Pakistani nuclear scientist widely recognized for his pivotal role in the development of Pakistan’s nuclear weapons program. Often referred to as the "father of the Pakistani bomb," Khan became a prominent figure in global discussions about nuclear proliferation, largely due to his controversial activities in the international nuclear black market.'), Document(metadata={'source': 'documents.txt'}, page_content='### Early Life and Education:'), Document(metadata={'source': 'documents.txt'}, page_content="Khan was born on April 1, 1936, in Bhopal, British India, in a well-off Muslim family. After the partition of India in 1947, his family moved to Pakistan. He attended the University of Karachi, where he studied metallurgy. Khan later moved to Europe for advanced studies, earning a degree in metallurgical engineering from the Technical University of Berlin, Germany, in the early 1960s. He al

In [ ]:
from langchain.vectorstores import Pinecone


retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


retriever = vector_store.as_retriever()


<ipython-input-7-b373e6fbbf23>:4: LangChainDeprecationWarning: The class `Pinecone` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-pinecone package and should be used instead. To use it run `pip install -U :class:`~langchain-pinecone` and import as `from :class:`~langchain_pinecone import Pinecone``.
  retriever=  Pinecone(index=index, embedding=embeddings, text_key="text")


In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="map_reduce",
    retriever=retriever
)

In [ ]:

question = input("I am a langchain chatbot. you can ask me for abdul qadeer khan.\n ")
response = qa_chain.run(question)
print(response)


I am a langchain chatbot. you can ask me for abdul qadeer khan.
 who is khan
Khan was born on April 1, 1936, in Bhopal, British India. He studied metallurgy at the University of Karachi, and later earned degrees in metallurgical engineering and nuclear physics from universities in Germany and Belgium, respectively. He is known for his role in helping Pakistan develop nuclear weapons, and is considered a national hero in Pakistan. However, he is also known for his involvement in selling nuclear technology to other countries, including Iran, North Korea, and Libya. Khan passed away on October 10, 2021, due to complications from COVID-19.

